<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/transferencia_de_aprendizaje/xception_preentrenado_con_imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje (Transfer learning) y ajuste fino (fine-tuning)

**Objetivo:** Se busca proveer de una guía completa en el aprendizaje por transferencia y ajuste fino, mejor conocido como *transfer learning & fine-tuning* respectivamente tomando como ejemplo la arquitectura Xception.

## Librerías

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Introducción

**Transferir aprendizaje** consiste en tomar características aprendidas en un problema y
aprovechándolos en un nuevo problema similar. Por ejemplo, las características de un modelo que tiene
aprendido a identificar mapaches puede ser útil para poner en marcha un modelo destinado a identificar
  tanukis.

El aprendizaje de transferencia generalmente se realiza para tareas en las que su conjunto de datos tiene muy pocos datos para
  entrenar un modelo a escala real desde cero.

La encarnación más común del aprendizaje por transferencia en el contexto del aprendizaje profundo es el
  siguiente flujo de trabajo:

1. Tome capas de un modelo previamente entrenado.
2. Congelarlos, para evitar destruir la información que contienen durante
  futuras rondas de entrenamiento.
3. Agregue algunas capas nuevas que se puedan entrenar encima de las capas congeladas. Aprenderán a girar
  las características antiguas en predicciones sobre un nuevo conjunto de datos.
4. Entrene las nuevas capas en su conjunto de datos.

Un último paso opcional es el **ajuste fino**, que consiste en descongelar todo el
modelo que obtuvo arriba (o parte de él), y volver a entrenarlo en los nuevos datos con un
Tasa de aprendizaje muy baja. Esto puede potencialmente lograr mejoras significativas, al
  adaptando gradualmente las funciones preentrenadas a los nuevos datos.

Primero, repasaremos en detalle la API `entrenable` de Keras, que subyace a la mayoría
  transferir el aprendizaje y ajustar los flujos de trabajo.

Luego, demostraremos el flujo de trabajo típico tomando un modelo previamente entrenado en el
Conjunto de datos de ImageNet y volver a entrenarlo en la clasificación de Kaggle "gatos contra perros"
  conjunto de datos

Esto está adaptado de
[Aprendizaje profundo con Python](https://www.manning.com/books/deep-learning-with-python)
  y la publicación de blog de 2016
["construir poderosos modelos de clasificación de imágenes usando muy poco
  data"](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

## Congelar capas: entender el atributo `entrenable`

Las capas y los modelos tienen tres atributos de peso:

- `pesos` es la lista de todas las variables de peso de la capa.
- `trainable_weights` es la lista de aquellos que deben actualizarse (a través de gradiente
  descenso) para minimizar la pérdida durante el entrenamiento.
- `non_trainable_weights` es la lista de aquellos que no están destinados a ser entrenados.
  Por lo general, el modelo los actualiza durante el pase hacia adelante.

**Ejemplo: la capa `Densa` tiene 2 pesos entrenables (núcleo y sesgo)**

In [ ]:
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

En general, todos los pesos son pesos entrenables. La única capa integrada que tiene
pesos no entrenables es la capa `BatchNormalization`. Utiliza pesos no entrenables.
  para realizar un seguimiento de la media y la varianza de sus entradas durante el entrenamiento.
Para aprender a usar pesos que no se pueden entrenar en sus propias capas personalizadas, consulte la
[guía para escribir nuevas capas desde cero](https://keras.io/guides/making_new_layers_and_models_via_subclassing/).

**Ejemplo: la capa `BatchNormalization` tiene 2 pesos entrenables y 2 no entrenables
  pesos**

In [ ]:
layer = keras.layers.BatchNormalization()
layer.build((None, 4))  # Create the weights

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

Las capas y los modelos también cuentan con un atributo booleano `entrenable`. Su valor se puede cambiar.
Establecer `layer.trainable` en `False` mueve todos los pesos de la capa de entrenable a
no entrenable. Esto se llama "congelar" la capa: el estado de una capa congelada no
actualizarse durante el entrenamiento (ya sea cuando se entrena con `fit()` o cuando se entrena con
  cualquier bucle personalizado que dependa de `trainable_weights` para aplicar actualizaciones de gradiente).

**Ejemplo: establecer `trainable` en `False`**

In [ ]:
layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights
layer.trainable = False  # Freeze the layer

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

Cuando un peso entrenable se vuelve no entrenable, su valor ya no se actualiza durante el entrenamiento.

In [ ]:
# Make a model with 2 layers
layer1 = keras.layers.Dense(3, activation="relu")
layer2 = keras.layers.Dense(3, activation="sigmoid")
model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

# Freeze the first layer
layer1.trainable = False

# Keep a copy of the weights of layer1 for later reference
initial_layer1_weights_values = layer1.get_weights()

# Train the model
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# Check that the weights of layer1 have not changed during training
final_layer1_weights_values = layer1.get_weights()
np.testing.assert_allclose(
    initial_layer1_weights_values[0], final_layer1_weights_values[0]
)
np.testing.assert_allclose(
    initial_layer1_weights_values[1], final_layer1_weights_values[1]
)

No confundir el atributo `layer.trainable` con el argumento `training` en `layer.__call__()` (que controla si la capa debe ejecutar su paso hacia adelante en modo de inferencia o modo de entrenamiento). Para obtener más información, consulte [Preguntas frecuentes de Keras](https://keras.io/getting_started/faq/#whats-the-difference-between-the-training-argument-in-call-and-the-trainable-attribute).

## Configuración recursiva del atributo `entrenable`

Si establece `trainable = False` en un modelo o en cualquier capa que tenga subcapas,
todas las capas de niños también se vuelven no entrenables.

**Ejemplo:**

In [ ]:
inner_model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        keras.layers.Dense(3, activation="relu"),
        keras.layers.Dense(3, activation="relu"),
    ]
)

model = keras.Sequential(
    [keras.Input(shape=(3,)), inner_model, keras.layers.Dense(3, activation="sigmoid"),]
)

model.trainable = False  # Freeze the outer model

assert inner_model.trainable == False  # All layers in `model` are now frozen
assert inner_model.layers[0].trainable == False  # `trainable` is propagated recursively

## El típico flujo de trabajo de aprendizaje por transferencia

Esto nos lleva a cómo se puede implementar un flujo de trabajo de aprendizaje de transferencia típico en Keras:

1. Cree una instancia de un modelo base y cargue pesos previamente entrenados en él.
2. Congele todas las capas en el modelo base configurando `trainable = False`.
3. Cree un nuevo modelo encima de la salida de una (o varias) capas de la base
  modelo.
4. Entrene su nuevo modelo en su nuevo conjunto de datos.

Tenga en cuenta que un flujo de trabajo alternativo y más ligero también podría ser:

1. Cree una instancia de un modelo base y cargue pesos previamente entrenados en él.
2. Ejecute su nuevo conjunto de datos y registre la salida de una (o varias) capas del modelo base. Esto se llama **extracción de características**.
3. Use esa salida como datos de entrada para un modelo nuevo y más pequeño.

Una ventaja clave de ese segundo flujo de trabajo es que solo ejecuta el modelo base una vez en
  sus datos, en lugar de una vez por época de entrenamiento. Así que es mucho más rápido y más barato.

Sin embargo, un problema con ese segundo flujo de trabajo es que no le permite dinámicamente
modifique los datos de entrada de su nuevo modelo durante el entrenamiento, lo cual es necesario al hacer
aumento de datos, por ejemplo. El aprendizaje por transferencia se usa típicamente para tareas cuando
su nuevo conjunto de datos tiene muy pocos datos para entrenar un modelo a gran escala desde cero, y en
tales escenarios el aumento de datos es muy importante. Así que en lo que sigue, nos centraremos
  en el primer flujo de trabajo.

Así es como se ve el primer flujo de trabajo en Keras:

Primero, crea una instancia de un modelo base con pesos previamente entrenados.

```python
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
```

Luego, congela el modelo base.

```python
base_model.trainable = False
```

Crea un nuevo modelo.

```python
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
```

Entrenar el modelo en datos nuevos.

```python
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)
```

## Sintonia FINA

Una vez que su modelo haya convergido en los nuevos datos, puede intentar descongelar todo o parte de
  el modelo base y volver a entrenar todo el modelo de extremo a extremo con una tasa de aprendizaje muy baja.

Este es un último paso opcional que potencialmente puede brindarle mejoras incrementales.
  También podría conducir a un sobreajuste rápido, tenlo en cuenta.

Es fundamental realizar este paso solo *después* de que el modelo con capas congeladas haya sido
capacitados para la convergencia. Si mezcla capas entrenables inicializadas aleatoriamente con
Capas entrenables que contienen funciones preentrenadas, las capas inicializadas aleatoriamente
causar actualizaciones de gradiente muy grandes durante el entrenamiento, lo que destruirá su entrenamiento previo
  características.

También es fundamental usar una tasa de aprendizaje muy baja en esta etapa, porque
está entrenando un modelo mucho más grande que en la primera ronda de entrenamiento, en un conjunto de datos
  eso es típicamente muy pequeño.
Como resultado, corre el riesgo de sobreajustarse muy rápidamente si aplica mucho peso.
  actualizaciones Aquí, solo desea readaptar los pesos preentrenados de forma incremental.

Así es como se implementa el ajuste fino de todo el modelo base:

```python
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

# Train end-to-end. Be careful to stop before you overfit!
model.fit(new_dataset, epochs=10, callbacks=..., validation_data=...)
```

**Nota importante sobre `compile()` y `entrenable`**

Llamar a `compile()` en un modelo está destinado a "congelar" el comportamiento de ese modelo. Esto
  implica que el `entrenable`
Los valores de los atributos en el momento en que se compila el modelo deben conservarse durante todo el
  toda la vida de ese modelo,
hasta que `compile` sea llamado de nuevo. Por lo tanto, si cambia cualquier valor 'entrenable', asegúrese de
  para llamar a `compile()` de nuevo en su
modelo para que sus cambios sean tomados en cuenta.

**Notas importantes sobre la capa `BatchNormalization`**

Muchos modelos de imagen contienen capas `BatchNormalization`. Esa capa es un caso especial en
  todos los conteos imaginables. Aquí hay algunas cosas a tener en cuenta.

- `BatchNormalization` contiene 2 pesos no entrenables que se actualizan durante
capacitación. Estas son las variables que siguen la media y la varianza de las entradas.
- Cuando establece `bn_layer.trainable = False`, la capa `BatchNormalization`
se ejecuta en modo de inferencia y no actualizará sus estadísticas de media y varianza. Esto no es
el caso de otras capas en general, como
[weight trainability & inference/training modes are two orthogonal concepts](
  https://keras.io/getting_started/faq/#whats-the-difference-between-the-training-argument-in-call-and-the-trainable-attribute).

Pero los dos están empatados en el caso de la capa `BatchNormalization`.
- Cuando descongela un modelo que contiene capas `BatchNormalization` para hacer
ajuste fino, debe mantener las capas `BatchNormalization` en modo de inferencia
  pasando `training=False` al llamar al modelo base.
De lo contrario, las actualizaciones aplicadas a los pesos no entrenables destruirán repentinamente
lo que el modelo ha aprendido.

Verá este patrón en acción en el ejemplo completo al final de esta guía.

## Transfiera el aprendizaje y ajuste con un ciclo de entrenamiento personalizado

Si en lugar de `fit()`, está utilizando su propio ciclo de entrenamiento de bajo nivel, el flujo de trabajo
permanece esencialmente igual. Debe tener cuidado de tener en cuenta solo la lista.
  `model.trainable_weights` al aplicar actualizaciones de gradiente:

```python
# Create base model
base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(150, 150, 3),
    include_top=False)
# Freeze base model
base_model.trainable = False

# Create new model on top.
inputs = keras.Input(shape=(150, 150, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()

# Iterate over the batches of a dataset.
for inputs, targets in new_dataset:
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        # Forward pass.
        predictions = model(inputs)
        # Compute the loss value for this batch.
        loss_value = loss_fn(targets, predictions)

    # Get gradients of loss wrt the *trainable* weights.
    gradients = tape.gradient(loss_value, model.trainable_weights)
    # Update the weights of the model.
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
```

Del mismo modo para el ajuste fino.

## Un ejemplo completo: ajuste fino de un modelo de clasificación de imágenes en un conjunto de datos de gatos contra perros

Para solidificar estos conceptos, lo guiaremos a través de una transferencia concreta de un extremo a otro
Ejemplo de aprendizaje y ajuste. Cargaremos el modelo Xception, pre-entrenado en
  ImageNet y utilícelo en el conjunto de datos de clasificación "gatos contra perros" de Kaggle.

### Obtener los datos

Primero, obtengamos el conjunto de datos de gatos contra perros usando TFDS. Si tiene su propio conjunto de datos,
probablemente querrás usar la utilidad
`tf.keras.utils.image_dataset_from_directory` para generar etiquetas similares
  objetos de conjunto de datos de un conjunto de imágenes en disco archivadas en carpetas específicas de clase.

Transferir el aprendizaje es más útil cuando se trabaja con conjuntos de datos muy pequeños. para mantener nuestro
conjunto de datos pequeño, usaremos el 40% de los datos de entrenamiento originales (25,000 imágenes) para
  capacitación, 10% para validación y 10% para prueba.

In [ ]:
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Estas son las primeras 9 imágenes en el conjunto de datos de entrenamiento; como puede ver, todas tienen diferentes tamaños.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis("off")

También podemos ver que la etiqueta 1 es "perro" y la etiqueta 0 es "gato".

### Estandarizando los datos

Nuestras imágenes en bruto tienen una variedad de tamaños. Además, cada píxel consta de 3 enteros
valores entre 0 y 255 (valores de nivel RGB). Esto no es muy adecuado para alimentar a un
  red neuronal Tenemos que hacer 2 cosas:

- Estandarizar a un tamaño de imagen fijo. Elegimos 150x150.
- Normalice los valores de píxel entre -1 y 1. Haremos esto usando una capa de `Normalización` como
  parte del propio modelo.

En general, es una buena práctica desarrollar modelos que toman datos sin procesar como entrada, como
a diferencia de los modelos que toman datos ya preprocesados. La razón es que, si su
el modelo espera datos preprocesados, cada vez que exporta su modelo para usarlo en otro lugar
(en un navegador web, en una aplicación móvil), deberá volver a implementar exactamente el mismo
tubería de preprocesamiento. Esto se vuelve muy complicado muy rápidamente. Así que deberíamos hacer lo mínimo
  posible cantidad de preprocesamiento antes de llegar al modelo.

Aquí, cambiaremos el tamaño de la imagen en la canalización de datos (porque una red neuronal profunda puede
procesar solo lotes contiguos de datos), y haremos la escala del valor de entrada como parte
  del modelo, cuando lo creamos.

Redimensionemos las imágenes a 150x150:

In [ ]:
size = (150, 150)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

Además, agrupemos los datos por lotes y utilicemos el almacenamiento en caché y la captación previa para optimizar la velocidad de carga.

In [ ]:
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

### Usar aumento de datos aleatorios

Cuando no tiene un conjunto de datos de imágenes grande, es una buena práctica introducir artificialmente diversidad de muestras aplicando transformaciones aleatorias pero realistas a las imágenes de entrenamiento, como cambios horizontales aleatorios o pequeñas rotaciones aleatorias. Esto ayuda a exponer el modelo a diferentes aspectos de los datos de entrenamiento mientras ralentiza el sobreajuste.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

Visualicemos cómo se ve la primera imagen del primer lote después de varias transformaciones aleatorias:

In [ ]:
import numpy as np

for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[0]))
        plt.axis("off")

## Construir un modelo

Ahora construyamos un modelo que siga el modelo que hemos explicado anteriormente.

Tenga en cuenta que:

- Agregamos una capa de `Rescaling` para escalar los valores de entrada (inicialmente en `[0, 255]`
  rango) al rango `[-1, 1]`.
- Agregamos una capa `Dropout` antes de la capa de clasificación, para la regularización.
- Nos aseguramos de pasar `training=False` al llamar al modelo base, para que
se ejecuta en modo de inferencia, por lo que las estadísticas de normas por lotes no se actualizan
incluso después de descongelar el modelo base para realizar ajustes.

In [ ]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

## Entrenamiento de modelo

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

## Haga una ronda de ajuste fino de todo el modelo

Finalmente, descongelemos el modelo base y entrenemos todo el modelo de principio a fin con una tasa de aprendizaje baja.

Es importante destacar que, aunque el modelo base se vuelve entrenable, todavía se ejecuta en modo de inferencia ya que pasamos `training=False` cuando lo llamamos cuando construimos el modelo. Esto significa que las capas de normalización de lotes internas no actualizarán sus estadísticas de lotes. Si lo hicieran, causarían estragos en las representaciones aprendidas por el modelo hasta el momento.

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.

base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Después de 10 épocas, el ajuste fino nos brinda una buena mejora aquí.